In [ ]:
import numpy as np
import pickle
import tensorflow as tf

In [ ]:
devices = tf.config.experimental.list_physical_devices("GPU")

In [ ]:
devices

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [ ]:
# lets try google embeddings according to the paper
!mkdir ~/.kaggle
!cp /content/drive/MyDrive/kaggle.json /root/.kaggle
!chmod 600 /root/.kaggle/kaggle.json
!kaggle datasets download alvations/vegetables-google-word2vec

100% 1.65G/1.66G [00:16<00:00, 106MB/s]
100% 1.66G/1.66G [00:16<00:00, 108MB/s]


In [ ]:
!unzip /content/vegetables-google-word2vec.zip

Archive:  /content/vegetables-google-word2vec.zip
  inflating: word2vec.news.negative-sample.300d.npy  
  inflating: word2vec.news.negative-sample.300d.pkl  
  inflating: word2vec.news.negative-sample.300d.txt  


In [ ]:
embedding=np.load('/content/word2vec.news.negative-sample.300d.npy')

In [ ]:
d_token=[line.strip() for line in open('/content/word2vec.news.negative-sample.300d.txt')]
d_token[5]

'on'

In [ ]:
query=str(input())
embedding_matrix=np.zeros((len(query.split()),300))

for i,word in enumerate(query.split()):
  if word in d_token:
    embedding_vector=embedding[d_token.index(word.lower())]
    embedding_matrix[i]=embedding_vector
  else:
    embedding_matrix[i]=np.array([0]*300)

who are you


In [ ]:
embedding_matrix.shape

(3, 300)

In [ ]:
score=sum(np.add.reduce(embedding_matrix,axis=1))/len(query.split())

In [ ]:
score

-0.4509042104085286

In [ ]:
import pandas as pd

In [ ]:
# for QA score to find cosine similarity between different questions and query
#calculate this score for every question present in the database(Quora question pairs)

df=pd.read_csv('/content/drive/MyDrive/Data/quora duplicate identification/train.csv')
df.head()


,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [ ]:
df=df.dropna()

In [ ]:
pd.DataFrame(df['question1'])

,question1
0,What is the step by step guide to invest in sh...
1,What is the story of Kohinoor (Koh-i-Noor) Dia...
2,How can I increase the speed of my internet co...
3,Why am I mentally very lonely? How can I solve...
4,"Which one dissolve in water quikly sugar, salt..."
...,...
404285,How many keywords are there in the Racket prog...
404286,Do you believe there is life after death?
404287,What is one coin?
404288,What is the approx annual cost of living while...


In [ ]:
x1_data=list(df['question1'].values)
x2_data=list(df['question2'].values)

In [ ]:
list(x1_data[:3])

['What is the step by step guide to invest in share market in india?',
 'What is the story of Kohinoor (Koh-i-Noor) Diamond?',
 'How can I increase the speed of my internet connection while using a VPN?']

In [ ]:
x1_data[len(x1_data):]=x2_data

In [ ]:
len(x1_data)

808574

In [ ]:
q_embedding_matrix=np.zeros((len(x1_data),300))
embedding_data=[]
question_embedding={}

In [ ]:
# some regex based cleaning
import re

def regex_clean(text):
  text=re.sub(r'[^\w\s]', '',text)               # texts without punctuation  \ used just in case if alpha-numeric characters has to be replaced for beteer replacements
  text=re.sub(r'[^A-Za-z0-9]+', ' ', text)       # texts contains only alpha-numeric characters
  text=re.sub('  ', '', text)                    # removing an extra spaces
  text=re.sub('   ','',text)                     # removing two extra spaces
  return text

In [ ]:
x_data=[regex_clean(ques) for ques in x1_data]

In [ ]:
for i,ques in enumerate(x_data[:10000]):
  # if s==0:
  #   quesbank=[q1,q2]
  #   for ques in quesbank:
  #     q=ques.split()
  #     for word in q:
  #       q_embedding_vector=embedding[d_token.index(word)]
  #       q_embedding[word]=np.array(q_embedding_vector)
  #     question.append(tuple(f'q{n}',q_embedding)
  #     q_embedding={}
  #     n+=1

  # else:
    q=ques.split()
    for word in q:
      if word in d_token:
        q_embedding_vector=embedding[d_token.index(word)]
        q_embedding_matrix[i]=q_embedding_vector
      else:
        q_embedding_matrix[i]=np.array([0]*300)
      embedding_data.append(q_embedding_matrix[i])
    question_embedding[ques]=np.array(embedding_data)
    embedding_data=[]
        
  

In [ ]:
question_embedding[x_data[0]]

array([[-0.234375  , -0.07177734,  0.01055908, ..., -0.09521484,
        -0.11621094, -0.11230469],
       [-0.234375  , -0.07177734,  0.01055908, ..., -0.09521484,
        -0.11621094, -0.11230469],
       [-0.234375  , -0.07177734,  0.01055908, ..., -0.09521484,
        -0.11621094, -0.11230469],
       ...,
       [-0.234375  , -0.07177734,  0.01055908, ..., -0.09521484,
        -0.11621094, -0.11230469],
       [-0.234375  , -0.07177734,  0.01055908, ..., -0.09521484,
        -0.11621094, -0.11230469],
       [-0.234375  , -0.07177734,  0.01055908, ..., -0.09521484,
        -0.11621094, -0.11230469]])

In [ ]:
d_score=sum(np.add.reduce(question_embedding[x_data[0]],axis=1)/len(x_data[0].split()))
d_score

-9.012670516967775

In [ ]:
np.add.reduce([[1,2],[1,2]])

array([2, 4])

In [ ]:
#cosine similarity check for first 10 ques
sim_score=[]
for i in range(len(x_data[:10])):
  d_score=sum(np.add.reduce(question_embedding[x_data[i]])/len(x_data[i].split()))
  sim=np.dot(score,d_score)
  sim_score.append(sim)

In [ ]:
sim_score

[4.063851083125579,
 1.601361506836838,
 3.1603268994076643,
 0.5772910038213013,
 1.180488952406146,
 0.5253270387923596,
 -0.0,
 -0.8289431095327018,
 -0.0,
 -0.0]